# Azure OpenAI and Semantic Kernel Tutorial

## Setup

In [ ]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.open_ai.services.azure_text_embedding import AzureTextEmbedding
from semantic_kernel.connectors.memory.azure_cognitive_search import (
    AzureCognitiveSearchMemoryStore,
)
from semantic_kernel.contents import ChatHistory
from semantic_kernel.core_plugins.http_plugin import HttpPlugin
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.sessions_python_tool.sessions_python_plugin import (
    SessionsPythonTool,
)
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

In [ ]:
# API endpoints
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = "gpt-4"
AZURE_OPENAI_ENDPOINT = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
AZURE_OPENAI_API_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXX"
AZURE_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada-002"
AZURE_EMBEDDING_ENDPOINT = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
# initialize kernel
kernel = Kernel()
print(kernel.services)

In [ ]:
# add deployments to kernel
chat_service_id = "gpt-4"
azure_openai_chat_gpt35 = AzureChatCompletion(
    service_id=chat_service_id,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)
embedding_service_id = "text-embedding-ada-002"
embedding_gen = AzureTextEmbedding(
    service_id=embedding_service_id,
    deployment_name=AZURE_EMBEDDING_DEPLOYMENT_NAME,
    endpoint=AZURE_EMBEDDING_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
)
kernel.add_service(azure_openai_chat_gpt35)
kernel.add_service(embedding_gen)
print(kernel.services)

## Semantic Functions, Prompts, Embeddings

In [ ]:
# process:
# 1. prompt
# 2. execution settings
# 3. prompt template config (prompt + execution settings + inputs)
# 4. add function (prompt template) to kernel
# 5. invoke kernel function

In [ ]:
# prompt
simple_prompt = """
You are an expert chat bot in answering questions and giving answers.
###INSTRUCTIONS###
***BE CONCISE, LESS IS MORE, USE 1 SENTENCE IF POSSIBLE***
###OUTPUT FORMAT###
Answer: ...
+++++
Answer the following in the style of {{$style}}: 
Question: {{$input}}
+++++
"""

In [ ]:
# execution settings - option 1 - dictionary
simple_execution_settings = {"default": {"max_tokens": 1000, "temperature": 0.7, "top_p": 0.6}}
# execution settings - option 2 - AzureChatPromptExecutionSettings
simple_execution_settings = AzureChatPromptExecutionSettings(
    service_id=chat_service_id, max_tokens=1000, temperature=0.7, top_p=0.8
)
# execution settings - option 3 - from service id
simple_execution_settings = kernel.get_prompt_execution_settings_from_service_id(chat_service_id)
simple_execution_settings.max_tokens = 1000
simple_execution_settings.temperature = 0.7
simple_execution_settings.top_p = 0.8

In [ ]:
# prompt template (prompt, inputs, and execution settings)
simple_prompt_template_config = PromptTemplateConfig(
    template=simple_prompt,
    name="example-prompt-config",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="user input text", is_required=True),
        InputVariable(
            name="style",
            description="answer style",
            default="professional",
            is_required=False,
        ),
    ],
    execution_settings=simple_execution_settings,
)

In [ ]:
# add function to kernel
question_function = kernel.add_function(
    function_name="my_simple_question_function",
    plugin_name="my_simple_question_plugin",
    prompt_template_config=simple_prompt_template_config,
)

In [ ]:
# run
input_text = "Who is David Lee Roth?"
style_text = "A Surfer Dude"
result = await kernel.invoke(question_function, KernelArguments(input=input_text, style=style_text))
print(result)

In [ ]:
# get multiple responses
multiple_execution_settings = simple_execution_settings.copy()
multiple_execution_settings.number_of_responses = 3
results = await azure_openai_chat_gpt35.get_text_contents(
    prompt=input_text, settings=multiple_execution_settings
)
for i, each in enumerate(results):
    print(i + 1, each)

In [ ]:
# get text embedding
await embedding_gen.generate_embeddings("David Lee Roth")

## Chat History

In [ ]:
# prompt with input and chat history
history_prompt = """
You are a useful chat bot, be as helpful as possible to the user.
###INSTRUCTIONS###
***BE CONCISE, LESS IS MORE, USE 1 SENTENCE IF POSSIBLE***
###CHAT HISTORY###
Use the following chat history if useful: 
{{$history}}
+++++
Answer the following: 
Question: {{$input}}
+++++
"""
history_execution_settings = AzureChatPromptExecutionSettings(
    service_id=chat_service_id,
    ai_model_id="gpt-4",
    max_tokens=2000,
    temperature=0.7,
)
history_prompt_template_config = PromptTemplateConfig(
    template=history_prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="user input", is_required=True),
        InputVariable(name="history", description="conversation history", is_required=True),
    ],
    execution_settings=history_execution_settings,
)
history_chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=history_prompt_template_config,
)

In [ ]:
chat_history = ChatHistory()

In [ ]:
async def chat(input, chat_history):
    answer = await kernel.invoke(
        history_chat_function,
        KernelArguments(input=input, history=chat_history),
    )
    print(f"Chatbot: {answer}")
    chat_history.add_user_message(input)
    chat_history.add_assistant_message(str(answer))

In [ ]:
await chat("How many members have been in the band?", chat_history)

In [ ]:
await chat("The band is Van Halen", chat_history)

In [ ]:
await chat("What instruments did each person play?", chat_history)

In [ ]:
await chat("What about Gary Cherone?", chat_history)

In [ ]:
print(chat_history)

## Memory, Vector Database Search

In [ ]:
# temp memory store
memory = SemanticTextMemory(storage=VolatileMemoryStore(), embeddings_generator=embedding_gen)

In [ ]:
# add plugin
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

In [ ]:
# add information
collection_id = "temp"
await memory.save_information(collection=collection_id, id="info1", text="Your brother is Mike")
await memory.save_information(collection=collection_id, id="info2", text="Your sister is Lisa")
await memory.save_information(collection=collection_id, id="info3", text="You made $100 in 2010")
await memory.save_information(collection=collection_id, id="info4", text="You made $500 in 2020")
await memory.save_information(collection=collection_id, id="info4", text="ND stands for Notre Dame")

In [ ]:
# lookup information
result = await memory.get("temp", "info1")
result.text

In [ ]:
# search
questions = [
    "Who is my sister?",
    "Who is my brother?",
    "How much money did I make?",
    "What is today's date?",
    "Notre Dame",
]
for question in questions:
    print(f"Question: {question}")
    result = await memory.search(collection_id, question, limit=3, min_relevance_score=0.8)
    for each_result in result:
        print(each_result.text)
    print()

In [ ]:
# prompt with input and memory function (recall), will search for input in memory as context
memory_prompt = """
You are a useful chat bot, be as helpful as possible to the user.
###INSTRUCTIONS###
***BE CONCISE, LESS IS MORE, USE 1 SENTANCE IF POSSIBLE***
***DO NOT ANSWER QUESTION IF YOU ARE NOT 100% SURE***
###MEMORY###
Use the following information if useful: 
Context: {{recall $input}}
Context: {{recall 'Notre Dame'}}
+++++
Answer the following: 
Question: {{$input}}
+++++
"""
memory_prompt_template_config = PromptTemplateConfig(
    template=memory_prompt,
    execution_settings={
        chat_service_id: kernel.get_service(chat_service_id).get_prompt_execution_settings_class()(
            service_id=chat_service_id
        )
    },
)
chat_with_memory = kernel.add_function(
    function_name="chat_with_memory",
    plugin_name="chat",
    prompt_template_config=memory_prompt_template_config,
)

In [ ]:
answer = await kernel.invoke(
    chat_with_memory,
    KernelArguments(input="Who are the members of my family?"),
    collection="temp",
    limit=3,
    min_relevance_score=0.8,
)
print(answer)

In [ ]:
answer = await kernel.invoke(
    chat_with_memory,
    KernelArguments(input="How much money have I earned?"),
    collection="temp",
    limit=3,
    min_relevance_score=0.5,
)
print(answer)

In [ ]:
answer = await kernel.invoke(
    chat_with_memory,
    KernelArguments(input="What is ND?"),
    collection="temp",
    limit=3,
    min_relevance_score=0.5,
)
print(answer)

## Plugins

In [ ]:
# add plugins
text_plugin = kernel.add_plugin(plugin=TextPlugin(), plugin_name="TextPlugin")
http_plugin = kernel.add_plugin(plugin=HttpPlugin(), plugin_name="HTTPPlugin")
time_plugin = kernel.add_plugin(plugin=TimePlugin(), plugin_name="TimePlugin")
math_plugin = kernel.add_plugin(plugin=MathPlugin(), plugin_name="MathPlugin")

In [ ]:
# view functions
plugins_functions = {}
for plugin_name, plugin in kernel.plugins.items():
    functions = []
    for function_name, function in plugin.functions.items():
        functions.append(function_name)
    plugins_functions[plugin_name] = functions
for each_key, each_value in plugins_functions.items():
    print(f"{each_key}: {', '.join(each_value)}")

In [ ]:
# call function
result = await kernel.invoke(plugin_name="TimePlugin", function_name="now")
print(result)

## External Plugins

In [ ]:
# plugins are prompt-templates saved to the following external files:
# ./plugin_name/function_name/config.json <- settings
# ./plugin_name/function_name/skprompt.txt <- prompt

In [ ]:
# load plugin
plugin_functions = kernel.add_plugin(parent_directory="./", plugin_name="my_plugin")

In [ ]:
# load function
question_function = plugin_functions["my_function"]

In [ ]:
# run
result = await kernel.invoke(question_function, input="Where was Eddie Van Halen born?")
print(result)

## Native Functions

In [ ]:
# define class with functions using @kernel_function decorator
class CheckWeatherPlugin:
    """
    Checks the weather
    """

    @kernel_function(
        description="Checks the weather",
        name="CheckWeather",
    )
    def check_weather_location(self, location: str) -> str:
        """
        Checks weather
        Args:
            location: name of city
        Returns:
            weather
        """
        if location.lower() == "chicago":
            return "Sunny and 88 Degrees"
        elif location.lower() == "new york city":
            return "Snowing and 20 Degrees"
        else:
            return "I don't know"

In [ ]:
# test functions
test_object = CheckWeatherPlugin()
print(test_object.check_weather_location("chicago"))

In [ ]:
# test native functions
check_weather_plugin = kernel.add_plugin(CheckWeatherPlugin(), "CheckWeatherPlugin")
check_weather_function = check_weather_plugin["CheckWeather"]
result = await check_weather_function(kernel, location="new york city")
print(result)

## Auto Function Calling

In [ ]:
auto_prompt = """
You are a useful chat bot, be as helpful as possible to the user.
###CHAT HISTORY###
Use the following chat history if useful: {{$history}}
User: {{$input}}
"""
auto_execution_settings = AzureChatPromptExecutionSettings(
    service_id=chat_service_id,
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    function_choice_behavior=FunctionChoiceBehavior.Auto(auto_invoke=True),
)
auto_execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto(
    auto_invoke=True,
    filters={"included_plugins": ["CheckWeatherPlugin"]},
)
auto_prompt_template_config = PromptTemplateConfig(
    template=auto_prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="user input", is_required=True),
        InputVariable(name="history", description="conversation history", is_required=True),
    ],
    execution_settings=auto_execution_settings,
)
auto_chat_function = kernel.add_function(
    function_name="auto_chat",
    plugin_name="auto_chatPlugin",
    prompt_template_config=auto_prompt_template_config,
)

In [ ]:
chat_history = ChatHistory()

In [ ]:
async def auto_chat(input, chat_history):
    answer = await kernel.invoke(
        auto_chat_function, KernelArguments(input=input, history=chat_history)
    )
    print(f"Chatbot: {answer}")
    chat_history.add_user_message(input)
    chat_history.add_assistant_message(str(answer))
    return answer

In [ ]:
result = await auto_chat("Can you check the weather in Chicago?", chat_history)

In [ ]:
result = await auto_chat("Can you check the weather in New York City?", chat_history)

In [ ]:
result = await auto_chat("Can you check the weather in Hawaii?", chat_history)